In [1]:
pip install -r "../../requirement.txt"

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
import os
dirname = os.path.abspath('')
pardir = os.path.abspath(os.path.join(dirname, os.pardir))    
sys.path.append(os.path.join(dirname, 'tools/'))

from tools.DatasetExtraction import extract
import tools.UsefulFunctions as uf
import torch.nn as nn
import optuna
from optuna.trial import TrialState
import torch.optim as optim
import torch.utils.data
from torch.utils.data import DataLoader, TensorDataset

In [3]:
#Constants

RANDOM_STATE = 42
VAL_SIZE = 0.2

## Data Preparation

In [4]:
#Extract dataset

name = 'big_dataset'

X, yID, dataset = extract(name)

dataset loaded
features loaded
labels loaded


In [5]:
#Split the dataset

Xs, ys, datasets = uf.split(X, yID, dataset, VAL_SIZE, RANDOM_STATE)

X_train, X_val = Xs
y_train, y_val = ys
dataset_train, dataset_val = datasets

In [6]:
#Encode labels and move to available GPU

classes = ('CMAES', 'PSO', 'Adam', 'LBFGSB', 'RandomSearch')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

y_train_enc, y_val_enc = uf.encode_y(ys, classes, device)

In [7]:
#Preprocess and scale features

X_train, X_val = uf.preprocess_X([X_train, X_val], device)

scaler = uf.TorchStandardScaler()
scaler.fit(X_train[:,:32])

X_train, X_val = uf.scale([X_train, X_val], scaler)

In [8]:
#Create tensor datasets

train_set = TensorDataset(X_train, y_train_enc)
val_set = TensorDataset(X_val, y_val_enc)

## Hyperparameter tuning with Optuna

In [9]:
#Tuning functions

def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio.
    
    n_layers = trial.suggest_int("n_layers", 3, 10)  
    n_hidden_units = trial.suggest_int("n_hidden_units", 4, 1000)
    k_dropout = trial.suggest_float("k_dropout", 0, 1)
    
    model = uf.Net(n_layers, n_hidden_units, k_dropout)

    return model

def objective(trial):
    # Generate the model.
    model = define_model(trial).to(device)
    
    lr = trial.suggest_float("lr", 1e-6, 1e-1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-1, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    criterion = nn.CrossEntropyLoss()
    criterion = criterion.to(device)
    
    batch_size = trial.suggest_int("batch_size", 32, 2048, log=True)
    
    train_loader_hype = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader_hype = DataLoader(val_set, batch_size=batch_size)
    
    epochs = 100
    
    early_stopper = uf.EarlyStopper(10, 1)
    
    # Training of the model.
    for epoch in range(epochs):
        model.train()
        uf.train(train_loader_hype, model, optimizer, criterion)

        # Validation of the model.
        model.eval()
        loss, accuracy = uf.test(val_loader_hype, model, criterion)
        
        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if early_stopper.early_stop(accuracy):
            break
        
    return accuracy

In [10]:
#Tuning run

study = optuna.create_study(direction="maximize")
study.optimize(objective, timeout=5*3600, show_progress_bar=True)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-01-26 22:01:09,242] A new study created in memory with name: no-name-d022e4ea-054a-401a-b148-ae133d055d64


   0%|          | 00:00/5:00:00

[W 2024-01-26 22:01:18,892] Trial 0 failed with parameters: {'n_layers': 8, 'n_hidden_units': 260, 'k_dropout': 0.8165412259536566, 'lr': 0.000240140189237824, 'weight_decay': 0.035908929339923096, 'batch_size': 667} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\leon0\Capstone\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\leon0\AppData\Local\Temp\ipykernel_12908\2917186360.py", line 37, in objective
    uf.train(train_loader_hype, model, optimizer, criterion)
  File "C:\Users\leon0\Desktop\Engineering with AI\Capstone\Repository\CapstoneAI_TU_2023-2024\Scripts\DL\tools\UsefulFunctions.py", line 159, in train
    optimizer.zero_grad()
  File "C:\Users\leon0\Capstone\lib\site-packages\torch\_compile.py", line 24, in inner
    return torch._dynamo.disable(fn, recursive)(*args, **kwargs)
  File "C:\Users\leon0\Capstone\lib\site-packages\torch\_dynamo\eval_fr

KeyboardInterrupt: 